In [4]:
import json
import os
import csv

In [38]:
# 定义函数来删除指定属性
def remove_keys(json_data, keys_to_remove):
    if isinstance(json_data, dict):
        for key in keys_to_remove:
            json_data.pop(key, None)
        for key, value in json_data.items():
            if isinstance(value, (dict, list)):
                remove_keys(value, keys_to_remove)
    elif isinstance(json_data, list):
        for item in json_data:
            remove_keys(item, keys_to_remove)

In [39]:
input_folder_path = 'npm_OSV'

# 用于保存处理后的文件
output_folder_path = 'processed_OSV'
os.makedirs(output_folder_path, exist_ok=True)

# 获取原始文件夹中所有JSON文件
json_files = [f for f in os.listdir(input_folder_path) if f.endswith('.json')]

# 定义要删除的属性列表
keys_to_remove = ["references", "severity", "aliases"]

# 处理每个JSON文件
for json_file in json_files:
    # 构建完整的输入文件路径
    input_file_path = os.path.join(input_folder_path, json_file)

    # 读取JSON文件
    with open(input_file_path, 'r') as file:
        data = json.load(file)

    # 删除属性
    remove_keys(data, keys_to_remove)

    # 构建完整的输出文件路径
    output_file_path = os.path.join(output_folder_path, json_file)

    # 保存处理后的JSON文件到新文件夹
    with open(output_file_path, 'w') as file:
        json.dump(data, file, indent=4)

print("JSON文件处理完成，并已保存到", output_file_path)

JSON文件处理完成，并已保存到 processed_OSV/GHSA-c7m7-4257-h698.json


In [40]:
# 创建文件夹，保存 "affected"和”id“
output_folder_path = 'affected'

os.makedirs(output_folder_path, exist_ok=True)

input_folder_path = 'processed_OSV'

# 获取processed_OSV中所有JSON文件
json_files = [f for f in os.listdir(input_folder_path) if f.endswith('.json')]

for json_file in json_files:
    
    input_file_path = os.path.join(input_folder_path, json_file)

    # 读取JSON文件
    with open(input_file_path, 'r') as file:
        data = json.load(file)

    # 提取 "affected" 和 "id" 的值
    affected_data = {
        "id": data["id"],
        "affected": data["affected"]
    }

    # 构建完整的输出文件路径
    output_file_path = os.path.join(output_folder_path, json_file)

    # 保存提取的数据到新文件夹
    with open(output_file_path, 'w') as file:
        json.dump(affected_data, file, indent=4)

print("数据提取并已保存到", output_folder_path)

数据提取并已保存到 affected


### 生成osv_affected表

In [13]:
input_folder_path = 'affected'
output_csv_path = 'affected_data.csv'
json_files = [f for f in os.listdir(input_folder_path) if f.endswith('.json')]


with open(output_csv_path, 'w', newline='', encoding='utf-8') as csv_file:
    # 指定逗号为分隔符
    csv_writer = csv.writer(csv_file, delimiter=',')

    # 写入 CSV 文件的标题行
    csv_writer.writerow(["id", "package_name", "package_ecosystem", "package_purl", "ranges_type", "ranges_events_introduced", "ranges_events_fixed", "ranges_events_last_affected", "ranges_events_limit", "database_specific"])

    for json_file in json_files:
        # 构建完整的输入文件路径
        input_file_path = os.path.join(input_folder_path, json_file)

        # 读取 JSON 文件
        with open(input_file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)

        # 提取 "id" 和 "affected" 数据
        id_value = data["id"]

        affected_data = data["affected"]

        for affected_item in affected_data:
            package_name = affected_item["package"]["name"]
            package_ecosystem = affected_item["package"]["ecosystem"]
            package_purl = affected_item["package"]["purl"]

            for each_range in affected_item["ranges"]:
                ranges_type = each_range["type"]

                eventsList = each_range.get("events", [])

                for event in eventsList:
                    ranges_events_introduced = event.get("introduced", None)
                    ranges_events_fixed = event.get("fixed", None)
                    ranges_events_last_affected = event.get("last_affected", None)
                    ranges_events_limit = event.get("limit", None)

                # 处理 "database_specific" 的不同数据类型
                if isinstance(affected_item["database_specific"], str):
                    database_specific = affected_item["database_specific"]
                elif isinstance(affected_item["database_specific"], dict):
                    # 从字典中提取键值对，并将它们拼接成一个字符串
                    database_specific = "; ".join([f"{key}:{value}" for key, value in affected_item["database_specific"].items()])
                else:
                    database_specific = ""
print(f"数据已成功写入到 '{csv_file_path}' 文件")

数据已成功写入到 'affected_data.csv' 文件


### 生成osv_base表

In [25]:
input_folder_path = 'npm_OSV' # 完整数据

json_files = [f for f in os.listdir(input_folder_path) if f.endswith('.json')]

# 打开 CSV 文件以写入数据
with open('id_cweIds.csv', 'w', newline='') as cwe_csvfile:
    cwe_csv_writer = csv.writer(cwe_csvfile, delimiter=',')
    # 写入 CSV 文件的标题行
    cwe_csv_writer.writerow(["id", "cwe_id"])

with open('base_data.csv', 'w', newline='') as base_csvfile:
    base_csv_writer = csv.writer(base_csvfile, delimiter=',')
    base_csv_writer.writerow(["id", "summary", "details", "modified", "published", "severity", "schema_version"])

with open('id_aliases.csv', 'w', newline='') as aliases_csvfile:
    aliases_csv_writer = csv.writer(aliases_csvfile, delimiter=',')
    aliases_csv_writer.writerow(["id", "alias_id"])

with open('id_related.csv', 'w', newline='') as related_csvfile:
    related_csv_writer = csv.writer(related_csvfile, delimiter=',')
    related_csv_writer.writerow(["id", "related_id"])
    
for json_file in json_files:
    # 构建完整的输入文件路径
    input_file_path = os.path.join(input_folder_path, json_file)

    # 读取JSON文件
    with open(input_file_path, 'r') as file:
        data = json.load(file)
        
    id_value = data.get("id", None)
    summary = data.get("summary", None)
    details = data.get("details", None)
    modified = data.get("modified", None)
    published = data.get("published", None)
    severity = None
    aliases = data.get("aliases", []) # 漏洞别名，数组
    related = data.get("related", []) # 密切相关的漏洞id，数组，好像为空
    schema_version = data.get("schema_version", None)
    
    # cwe_ids：两部分来源
    # 1.affected[database_specific]里的[cwes][cweId](参考affected.csv)
    # 2.database_specific[cwe_ids] (数组)
    
    
    cwe_list = []
    # 判断 "database_specific" 是否存在
    if "database_specific" in data:
        db_specific = data["database_specific"]
        severity = db_specific.get("severity", None)

        # 判断 "cwe_ids" 是否存在
        if "cwe_ids" in db_specific:
            cwe_ids = db_specific["cwe_ids"]
            cwe_list.extend(cwe_ids) # 把cwe_ids列表里的元素添加到cwe_list里
            
                        
    # 检查 "affected" 是否存在
    if "affected" in data:
        affected_data = data["affected"]

        # 遍历 "affected" 数据
        for affected_item in affected_data:
            # 检查 "database_specific" 是否存在
            if "database_specific" in affected_item:
                database_specific = affected_item["database_specific"]

                # 检查 "cwes" 是否存在
                if "cwes" in database_specific:
                    cwes_data = database_specific["cwes"]

                    # 遍历 "cwes" 数组并提取每个 "cweId" 值
                    for cwe in cwes_data:
                        cwe_id = cwe["cweId"]
                        cwe_list.append(cwe_id)
    
    # 将cwe_list的值写到id_cweIds.csv中
    with open('id_cweIds.csv', 'a', newline='') as cwe_csvfile:
        cwe_csv_writer = csv.writer(cwe_csvfile, delimiter=',')
        for each_cwe_id in cwe_list:
            cwe_csv_writer.writerow([id_value, each_cwe_id])

    # 将aliases list的值写到id_aliases.csv中
    with open('id_aliases.csv', 'a', newline='') as aliases_csvfile:
        aliases_csv_writer = csv.writer(aliases_csvfile, delimiter=',')
        for each_alias in aliases:
            aliases_csv_writer.writerow([id_value, each_alias])

    # 将related list的值写到id_related.csv中
    with open('id_related.csv', 'a', newline='') as related_csvfile:
        related_csv_writer = csv.writer(related_csvfile, delimiter=',')
        for each_related in related:
            related_csv_writer.writerow([id_value, each_related])

    # 将其他字段写到base_data.csv中
    with open('base_data.csv', 'a', newline='') as base_csvfile:
        base_csv_writer = csv.writer(base_csvfile, delimiter=',')
        base_csv_writer.writerow([id_value, summary, details, modified, published, severity, schema_version])

print("成功写入id_cweIds.csv、base_data.csv、id_aliases.csv、id_related.csv文件")

成功写入id_cweIds.csv、base_data.csv、id_aliases.csv、id_related.csv文件
